## LangChain's

In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

import warnings
warnings.filterwarnings("ignore")

documents = TextLoader("state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, HuggingFaceEmbeddings()).as_retriever()

docs = retriever.get_relevant_documents(
    "What did the president say about Ketanji Brown Jackson?"
)

# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

pretty_print_docs(docs)

Document 1:

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

A former top litigator in private practice. A former federal public defender. And fro

In [35]:
from typing import Any, Optional
from uuid import UUID

from dotenv import load_dotenv
from langchain.callbacks.base import BaseCallbackHandler

from genai import Client, Credentials
from genai.extensions.langchain import LangChainInterface
from genai.text.generation import (
    DecodingMethod,
    ModerationHAP,
    ModerationParameters,
    TextGenerationParameters,
)

# make sure you have a .env file under genai root with
# GENAI_KEY=<your-genai-key>
# GENAI_API=<genai-api-endpoint> (optional) DEFAULT_API = "https://bam-api.res.ibm.com"
load_dotenv()


def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"


print(heading("Generate text with langchain"))


class Callback(BaseCallbackHandler):
    def on_llm_new_token(
        self,
        token: str,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> Any:
        print(f"Token received: {token}")


llm = LangChainInterface(
    # model_id="ibm/granite-13b-chat-v2",
    model_id='mistralai/mistral-7b-instruct-v0-2',
    client=Client(credentials=Credentials.from_env()),
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.SAMPLE,
        max_new_tokens=200,
        min_new_tokens=1,
        temperature=0.5,
        top_k=50,
        top_p=1,
    ),
    moderations=ModerationParameters(
        # Threshold is set to very low level to flag everything (testing purposes)
        # or set to True to enable HAP with default settings
        hap=ModerationHAP(input=True, output=True, threshold=0.01)
    ),
)

prompt = "Tell me about IBM."
print(f"Prompt: {prompt}")

result = llm.generate(prompts=[prompt], callbacks=[Callback()])

print(f"Answer: {result.generations[0][0].text}")
print(result.llm_output)
print(result.generations[0][0].generation_info)


========================= Generate text with langchain =========================

Prompt: Tell me about IBM.
Answer: 

IBM, or International Business Machines Corporation, is an American multinational technology company headquartered in Armonk, New York. It was founded in 1911 as the Tabulating Machine Company and was later renamed to International Business Machines in 1924. IBM is one of the oldest and largest information technology companies in the world.

IBM is known for its hardware, software, and consulting services. Its products and services include mainframe computers, servers, storage systems, networking equipment, software, and a wide range of consulting and business services. IBM also has a significant presence in the areas of cloud computing, artificial intelligence, and blockchain technology.

IBM has a global workforce of over 350,000 employees and operates in more than 170 countries. The company's revenue for 2020 was approximately $77 billion. IBM is listed on the New 

In [3]:
# import os
# from dotenv import load_dotenv
# from genai import Credentials, Client

# load_dotenv()
# api_key = os.getenv("GENAI_KEY", None)
# api_url = os.getenv("GENAI_API", None)
# creds = Credentials(api_key, api_endpoint=api_url)

In [4]:
# from genai.schema import TextGenerationParameters
# from genai import model

# credentials = Credentials.from_env()
# params = TextGenerationParameters(max_new_tokens=10, decoding_method='greedy', temperature=0.15)

# client = Client(credentials=credentials)
# responses = list(client.text.generation.create(model_id="google/flan-ul2", parameters=params, inputs=["What is IBM?"]))
# print(f"Generated Text: {responses[0].results[0].generated_text}")

In [5]:
# client.model.retrieve(id="google/flan-ul2")

In [6]:
context_list = [d.page_content for i, d in enumerate(docs)]
context_list, len(context_list)

(['Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
  'A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s

In [7]:
# The response from original prompt
# from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate

# llm = OpenAI(model="gpt-3.5-turbo-16k")

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n be very precise in answering"
)
question = "What did the president say about Ketanji Jackson Brown?"

qa_template = PromptTemplate(template)


# you can create text prompt (for completion API)
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

result = llm.generate(prompts=[prompt], callbacks=[Callback()])

print(f"Answer: {result.generations[0][0].text}")

# response = send_to_bam(prompts=[prompt], model_name="google/flan-ul2", decoding_method="greedy", max_new_tokens=200,
#                              min_new_tokens=30, temperature=1.0, repetition_penalty=1.0)

Answer:  the questions using only the given information.


In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
# from langchain_openai import OpenAI

# llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What did the president say about Ketanji Jackson Brown?"
)
pretty_print_docs(compressed_docs)

Document 1:

"Justice Stephen Breyer--an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court."
----------------------------------------------------------------------------------------------------
Document 2:

"Ketanji Jackson Brown has received a broad range of support--from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."
----------------------------------------------------------------------------------------------------
Document 3:

"We must hold social media platforms accountable for the national experiment they're conducting on our children." "Let's get all Americans the mental health services they need."
----------------------------------------------------------------------------------------------------
Document 4:

"I'm taking robust action to make sure the pain of our sanctions is targeted at Russia's economy." "We'll work together to hold Russia accountable," says President Biden in a statement rel

In [9]:

len("\n\n".join([d.page_content for i, d in enumerate(docs)]))

3565

In [10]:
original_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(docs)]))
compressed_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(compressed_docs)]))

print("Original context length:", original_contexts_len)
print("Compressed context length:", compressed_contexts_len)
print("Compressed Ratio:", f"{original_contexts_len/(compressed_contexts_len + 1e-5):.2f}x")


# Output: LLMChainExtractor ContextualCompressionRetriever
"""
Original context length: 3565
Compressed context length: 1183
Compressed Ratio: 3.01x
"""

Original context length: 3565
Compressed context length: 674
Compressed Ratio: 5.29x


'\nOriginal context length: 3565\nCompressed context length: 1183\nCompressed Ratio: 3.01x\n'

In [11]:
context_list = [d.page_content for i, d in enumerate(compressed_docs)]
context_list, len(context_list)

(['"Justice Stephen Breyer--an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court."',
  '"Ketanji Jackson Brown has received a broad range of support--from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."',
  '"We must hold social media platforms accountable for the national experiment they\'re conducting on our children." "Let\'s get all Americans the mental health services they need."',
  '"I\'m taking robust action to make sure the pain of our sanctions is targeted at Russia\'s economy." "We\'ll work together to hold Russia accountable," says President Biden in a statement released by his office today.'],
 4)

In [12]:
# The response from original prompt
# from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate

# llm = OpenAI(model="gpt-3.5-turbo-16k")

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n be very precise in answering"
)
question = "What did the president say about Ketanji Jackson Brown?"

qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

result = llm.generate(prompts=[prompt], callbacks=[Callback()])

print(f"Answer: {result.generations[0][0].text}")

# response = send_to_bam(prompts=[prompt], model_name="google/flan-ul2", decoding_method="greedy", max_new_tokens=200,
#                              min_new_tokens=30, temperature=1.0, repetition_penalty=1.0)

Answer:  the question with as much detail as possible.


In [13]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What did the president say about Ketanji Jackson Brown?"
)
pretty_print_docs(compressed_docs)

ValueError: BooleanOutputParser expected output value to either be YES or NO. Received Yes, because the context mentions "Justice Breyer" and "Ketanji Brown Jackson," which are both mentioned in the question. Additionally, the context also mentions "Circuit Court of Appeals Judge," which is related to the position being sought by Ketanji Brown Jackson..

In [ ]:

from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What did the president say about Ketanji Jackson Brown?"
)
pretty_print_docs(compressed_docs)

Document 1:

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [ ]:
original_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(docs)]))
compressed_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(compressed_docs)]))

print("Original context length:", original_contexts_len)
print("Compressed context length:", compressed_contexts_len)
print("Compressed Ratio:", f"{original_contexts_len/(compressed_contexts_len + 1e-5):.2f}x")


# Output: EmbeddingsFilter ContextualCompressionRetriever
"""
Original context length: 3565
Compressed context length: 787
Compressed Ratio: 4.53x
"""

Original context length: 3565
Compressed context length: 787
Compressed Ratio: 4.53x


'\nOriginal context length: 3565\nCompressed context length: 787\nCompressed Ratio: 4.53x\n'

In [ ]:
context_list = [d.page_content for i, d in enumerate(compressed_docs)]
context_list, len(context_list)

(['Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'],
 1)

In [ ]:
# The response from original prompt
# from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate

# llm = OpenAI(model="gpt-3.5-turbo-16k")

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n be very precise in answering"
)
question = "What did the president say about Ketanji Jackson Brown?"

qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

result = llm.generate(prompts=[prompt], callbacks=[Callback()])

print(f"Answer: {result.generations[0][0].text}")

# response = send_to_bam(prompts=[prompt], model_name="google/flan-ul2", decoding_method="greedy", max_new_tokens=200,
#                              min_new_tokens=30, temperature=1.0, repetition_penalty=1.0)

Answer:  my question


In [ ]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings = HuggingFaceBgeEmbeddings()
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What did the president say about Ketanji Jackson Brown?"
)
pretty_print_docs(compressed_docs)

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson


In [ ]:
original_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(docs)]))
compressed_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(compressed_docs)]))

print("Original context length:", original_contexts_len)
print("Compressed context length:", compressed_contexts_len)
print("Compressed Ratio:", f"{original_contexts_len/(compressed_contexts_len + 1e-5):.2f}x")


# Output: DocumentCompressorPipeline ContextualCompressionRetriever
"""
Original context length: 3565
Compressed context length: 237
Compressed Ratio: 15.04x
"""

Original context length: 3565
Compressed context length: 237
Compressed Ratio: 15.04x


'\nOriginal context length: 3565\nCompressed context length: 237\nCompressed Ratio: 15.04x\n'

In [ ]:
context_list = [d.page_content for i, d in enumerate(compressed_docs)]
context_list, len(context_list)

(['One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson'],
 1)

In [14]:
# The response from original prompt
# from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate

# llm = OpenAI(model="gpt-3.5-turbo-16k")

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n be very precise in answering"
)
question = "What did the president say about Ketanji Jackson Brown?"

qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

result = llm.generate(prompts=[prompt], callbacks=[Callback()])

print(f"Answer: {result.generations[0][0].text}")

# response = send_to_bam(prompts=[prompt], model_name="google/flan-ul2", decoding_method="greedy", max_new_tokens=200,
#                              min_new_tokens=30, temperature=1.0, repetition_penalty=1.0)

Answer:  the question. The correct response should include the following details:
- That the president said that Ketanji Jackson Brown had received strong support from various groups and individuals, including veterans and law enforcement officers.
- That the president also mentioned that she was an experienced judge who would bring valuable perspective to the court.
- That the president expressed confidence in her ability to serve as a justice on the Supreme Court.




## LLMLingua with LLamaIndex

In [15]:
from llama_index import (
    VectorStoreIndex,
    download_loader,
    load_index_from_storage,
    StorageContext,
)
# WikipediaReader = download_loader("WikipediaReader")
# loader = WikipediaReader()
# documents = loader.load_data(pages=['Mexican–American_War'])

In [16]:
# documents

In [17]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_dir='.', input_files=['state_of_the_union.txt']).load_data()
documents

[Document(id_='050c72d2-a93a-43f2-92bc-cde8f3fde250', embedding=None, metadata={'file_path': 'state_of_the_union.txt', 'file_name': 'state_of_the_union.txt', 'file_type': 'text/plain', 'file_size': 39027, 'creation_date': '2024-02-02', 'last_modified_date': '2024-01-30', 'last_accessed_date': '2024-02-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Cons

In [39]:
from llama_index import ServiceContext
from langchain.embeddings import HuggingFaceEmbeddings

service_context = ServiceContext.from_defaults(
    embed_model=HuggingFaceEmbeddings()#,
    # llm=llm
)

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

retriever = index.as_retriever(similarity_top_k=3)

In [40]:
# question = "What were the main outcomes of the war?"
question = "What did the president say about Ketanji Jackson Brown?"
contexts = retriever.retrieve(question)

In [41]:
context_list = [n.get_content() for n in contexts]
context_list, len(context_list)

(['Third, support our veterans. \n\nVeterans are the best of us. \n\nI’ve always believed that we have a sacred obligation to equip all those we send to war and care for them and their families when they come home. \n\nMy administration is providing assistance with job training and housing, and now helping lower-income veterans get VA care debt-free.  \n\nOur troops in Iraq and Afghanistan faced many dangers. \n\nOne was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. \n\nWhen they came home, many of the world’s fittest and best trained warriors were never the same. \n\nHeadaches. Numbness. Dizziness. \n\nA cancer that would put them in a flag-draped coffin. \n\nI know. \n\nOne of those soldiers was my son Major Beau Biden. \n\nWe don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. \n\nBut I’m committed to finding out everything we 

In [42]:
# The response from original prompt
# from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate

# llm = OpenAI(model="gpt-3.5-turbo-16k")

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n be very precise in answering"
)

qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

result = llm.generate(prompts=[prompt], callbacks=[Callback()])

print(f"Answer: {result.generations[0][0].text}")

# response = send_to_bam(prompts=[prompt], model_name="google/flan-ul2", decoding_method="greedy", max_new_tokens=200,
#                              min_new_tokens=30, temperature=1.0, repetition_penalty=1.0)

Answer:  the question and only address the specific topic.

The president did not mention Ketanji Jackson Brown in his State of the Union address.


In [43]:
retrieved_nodes = retriever.retrieve(question)
[n.get_content() for n in retrieved_nodes]


['Third, support our veterans. \n\nVeterans are the best of us. \n\nI’ve always believed that we have a sacred obligation to equip all those we send to war and care for them and their families when they come home. \n\nMy administration is providing assistance with job training and housing, and now helping lower-income veterans get VA care debt-free.  \n\nOur troops in Iraq and Afghanistan faced many dangers. \n\nOne was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. \n\nWhen they came home, many of the world’s fittest and best trained warriors were never the same. \n\nHeadaches. Numbness. Dizziness. \n\nA cancer that would put them in a flag-draped coffin. \n\nI know. \n\nOne of those soldiers was my son Major Beau Biden. \n\nWe don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. \n\nBut I’m committed to finding out everything we c

In [44]:
# Setup LLMLingua
from llama_index.indices.postprocessor import LongLLMLinguaPostprocessor

node_postprocessor = LongLLMLinguaPostprocessor(
    model_name='NousResearch/Llama-2-7b-hf', #'NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT',
    device_map='cpu',
    instruction_str="Given the context, please answer the final question",
    target_token=300,
    rank_method="longllmlingua",
    additional_compress_kwargs={
        "condition_compare": True,
        "condition_in_question": "after",
        "context_budget": "+100",
        "reorder_context": "sort",  # enable document reorder,
        "dynamic_context_compression_ratio": 0.3,
    },
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
from llama_index.indices.query.schema import QueryBundle

new_retrieved_nodes = node_postprocessor.postprocess_nodes(
    retrieved_nodes, query_bundle = QueryBundle(query_str=question)
)

In [46]:
new_context_list = [n.get_content() for n in new_retrieved_nodes]
new_context_list

['As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. ',
 'This was bipart effort, and I want to thank the members of both parties who worked to make it happen. ',
 'A that would put them in flag-draped coffin. ',
 'A cancer that would put them in a flag-draped coffin. \nThrough found purpose demand we better. ',
 'vide a pathway to citizenship Dreamers those on temporary status, workers, and essential workers. \n will meet the. ',
 'Let’s it once and for all. ',
 'As’ve told Xi Jinping, is never a American people. \nAR- will have—to drive breakthrough in Alz’s diabetes, and more. To protect freedom and, expandness opportunity \nLet me give of why’ so to pass it \nNow our to meet overcome challeng our time  Administration but are\nCommitted to DanielleWhenayer dollars America – going American products to, most nation theewation have critical in history to. ',
 'A

In [47]:
original_contexts = "\n\n".join([n.get_content() for n in retrieved_nodes])
compressed_contexts = "\n\n".join([n.get_content() for n in new_retrieved_nodes])
original_tokens = node_postprocessor._llm_lingua.get_token_length(original_contexts)
compressed_tokens = node_postprocessor._llm_lingua.get_token_length(compressed_contexts)
# print(compressed_contexts)
# print()
print("Original Tokens:", original_tokens)
print("Compressed Tokens:", compressed_tokens)
print("Compressed Ratio:", f"{original_tokens/(compressed_tokens + 1e-5):.2f}x")

Original Tokens: 3690
Compressed Tokens: 278
Compressed Ratio: 13.27x


In [48]:
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n be very precise in answering"
)

qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str="\n\n".join(new_context_list), query_str=question)

result = llm.generate(prompts=[prompt], callbacks=[Callback()])

print(f"Answer: {result.generations[0][0].text}")

# response = send_to_bam(prompts=[prompt], model_name="google/flan-ul2", decoding_method="greedy", max_new_tokens=200,
#                              min_new_tokens=30, temperature=1.0, repetition_penalty=1.0)

Answer: : The president expressed his support for Ketanji Jackson Brown, a transgender American, and acknowledged her potential. He mentioned that he had her back as her President and that she could be herself. He also thanked the members of both parties who worked to make it happen, likely referring to her confirmation as a Supreme Court justice. The president did not make any specific statements about her qualifications or judicial philosophy.


In [32]:
from llama_index.response_synthesizers import CompactAndRefine
synthesizer = CompactAndRefine()

response = synthesizer.synthesize(question, new_retrieved_nodes)
print(str(response))

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [35]:
def flexible_function(*args, **kwargs):
    print("Positional arguments: {}".format(args))
    print("Keyword arguments: {}".format(kwargs))

flexible_function(1, 2, 3, first='Alice', last='Bob')

Positional arguments: (1, 2, 3)
Keyword arguments: {'first': 'Alice', 'last': 'Bob'}


In [21]:
# This is a long document we can split up.
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()

In [22]:

from dotenv import load_dotenv

from genai.client import Client
from genai.credentials import Credentials

# make sure you have a .env file under genai root with
# GENAI_KEY=<your-genai-key>
# GENAI_API=<genai-api-endpoint>
load_dotenv()


def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"


client = Client(credentials=Credentials.from_env())

inputs = ["Hello", "world"]
model_id = "sentence-transformers/all-minilm-l6-v2"

print(heading("Running embedding for inputs in parallel"))
# yields batch of results that are produced asynchronously and in parallel
for input, response in zip(inputs, client.text.embedding.create(model_id=model_id, inputs=inputs)):
    print(f"Embedding vector for '{input}': {response.results}")


=================== Running embedding for inputs in parallel ===================

Embedding vector for 'Hello': [[-0.062771737575531, 0.05495883896946907, 0.05216483399271965, 0.08579006046056747, -0.0827488973736763, -0.07457299530506134, 0.06855472177267075, 0.01839638128876686, -0.08201129734516144, -0.03738482668995857, 0.012124953791499138, 0.0035183376166969538, -0.004134362563490868, -0.04378440976142883, 0.021807372570037842, -0.0051027266308665276, 0.019546614959836006, -0.042348723858594894, -0.11035960167646408, 0.005424540489912033, -0.05573483929038048, 0.02805246226489544, -0.023158734664320946, 0.028481375426054, -0.05370965227484703, -0.05260162800550461, 0.03393930569291115, 0.04538865014910698, 0.023718444630503654, -0.07312081754207611, 0.054777760058641434, 0.017047306522727013, 0.08136041462421417, -0.002862730761989951, 0.011958058923482895, 0.07355856895446777, -0.09423748403787613, -0.08136198669672012, 0.04001535475254059, 0.000692090718075633, -0.013393289409

In [24]:
state_of_the_union

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [23]:
from langchain_experimental.text_splitter import SemanticChunker
# from langchain_openai.embeddings import OpenAIEmbeddings

embedding = LangChainInterface(
    model_id="sentence-transformers/all-minilm-l6-v2",
    client=client)

text_splitter = SemanticChunker(embedding)

docs = text_splitter.create_documents([state_of_the_union])
print(docs[0].page_content)

AttributeError: 'LangChainInterface' object has no attribute 'embed_documents'